### 从 MongoDB 中获取数据并导出成 json 文件

In [2]:
from atomate.vasp.database import VaspCalcDb

db_json_fn = "db.json"
atomate_db = VaspCalcDb.from_db_file(db_json_fn)

In [3]:
query = {
    "tags.structure_id": "ICET-Training-No-0076857",
    "nsites": 8,
}

count = atomate_db.collection.count_documents(query)
count

4

In [4]:
results = atomate_db.collection.find(query)
results

In [5]:
type(results)  # 游标对象

pymongo.cursor.Cursor

In [6]:
import json

with open("output.json", "w") as file:
    # 将每个 document 转换为 JSON 格式并写入文件
    for result in results:
        # ensure_ascii=False 允许非 ASCII字符在字符串中直接显示
        # default=str 将不是 JSON 序列化的对象转换为字符串
        json_data = json.dumps(result, ensure_ascii=False, default=str)
        file.write(json_data + "\n")


In [7]:
with open("output.json", "r") as file:
    for line in file:
        # 读取每一行，并将其转换为字典
        data = json.loads(line)
        print(data)


{'_id': '654b02c2218345934ce20bcc', 'dir_name': 'node125.pi.sjtu.edu.cn:/dssg/home/acct-mseljf/mseljf-wm/ht/8-TiAlNbMoZr/block_2023-11-08-00-26-30-877502/launcher_2023-11-08-03-37-39-432417', 'analysis': {'delta_volume': -2.366771873702163, 'delta_volume_as_percent': -1.6315952297112932, 'max_force': 0.49342177063347015, 'warnings': ['Volume change > 20.0%'], 'errors': []}, 'calcs_reversed': [{'vasp_version': '6.2.1', 'has_vasp_completed': True, 'nsites': 8, 'elements': ['Al', 'Nb'], 'nelements': 2, 'run_type': 'GGA', 'input': {'incar': {'PREC': 'Accurate', 'ALGO': 'Normal', 'ADDGRID': False, 'ISPIN': 1, 'NELM': 100, 'NELMIN': 4, 'IBRION': 2, 'EDIFF': 1e-06, 'EDIFFG': 0.0001, 'NSW': 100, 'ISIF': 7, 'ENCUT': 350.0, 'POTIM': 0.25, 'MAGMOM': [0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6], 'LREAL': False, 'ISMEAR': 1, 'SIGMA': 0.1, 'LMAXMIX': 4, 'LWAVE': False, 'LORBIT': 11, 'LASPH': True}, 'kpoints': {'comment': 'Kpoints from vasprun.xml', 'nkpoints': 0, 'generation_style': 'Gamma', 'kpoints': 

循环遍历游标（Cursor）时，游标会随着每次迭代逐步前进，直到遍历完所有文档。游标遍历完毕后，它的位置会停留在结果集的末尾。因此，如果尝试再次遍历同一个游标，循环体内的代码将不会执行，因为游标已经处于结果集的末尾，相当于已经"耗尽"了。

In [8]:
for result in results:
    print(result)

将 results 存储在列表中，可能会占用较多的内存，尤其是当处理大量数据时。这种方法在数据集合较小的情况下效果最好。

In [9]:
results = atomate_db.collection.find(query)
results_list = list(results)
len(results_list)

4

In [10]:
# find_one() 返回查询结果的第一个文档
result_one = atomate_db.collection.find_one(query)
type(result_one)

dict